# Prodweek Conflict Matrix Generator
This is to help create prodweek schedule for cleaning / staging. Outputs directly onto Prodweek Conflict Form spreadsheet.\

**BEFORE YOU START:**
1. make sure to share the conflict sheets with ur scripts acc (example: hanjie@adt-scripts.iam.gserviceaccount.com)
2. make sure you have a client file (example: client_secret.json) in the directory

**TIMES TO RUN:**
1. Night that prodweek conflict form is due

## Defaults
Edit values!

1. `sheet_name` name of the Prodweek Conflict form responses
2. `response_worksheet_name` name of the worksheet that we want to work off of (will either be the first sheet or an edited version that removed duplicate submissions, bad excuses, etc)
3. `create_new_worksheet` True if the named worksheet does not already exist (will create new worksheets). False if the named worksheet already exists (will update the existing).
4. `named` True to append a suffix to generated worksheet names. False to use just the default ['cleaning conflicts matrix', 'cleaning excuses matrix', 'staging conflicts matrix', 'staging excuses matrix']
5. `name_of_run` suffix to attatch to worksheet name. example: 'cleaning conflicts matrix (edited)'


In [50]:
# Orignal Code author: cindywang'22, Spring 2022
# Notebook author: j13'24, Fall 2023
# Last updated: j13'24, Fall 2023

sheet_name = "[Fall 2023] Prod Week Conflicts (Responses)"
response_worksheet_name = 'edited responses'
create_new_worksheet = True
named = True
name_of_run = "(edited)"

1. `dance_list` a string of dance names from the Prodweek Conflict form split by tab values or \t.
2. `dances` a list of dance names.

For ease you can copy the Master Roster spreadsheet heading, which will give you a string for dance_list. You can input dance names not included either directly into the string (add \t so it can be splitted), or append it later to the dances list. 

**Debugging note: Dance names should match the Prodweek Conflict form EXACTLY, and include ALL dances on the form**

In [16]:
# copied from master roster for ease
dance_list = 'Exec Interlude\tSuper	Dreams of Dai	90s Love	Youth	The Eve	AYAYAYA	A Rainy Dawn	Secret	Spirit of Ink	PINKCHELLA	Big Fish	VENI VIDI VICI	Autumn Ripples	Butterfly	Broken Melodies	The Call of the Past	Teddy Bear	Blue'
dances = dance_list.split('\t')
dances

['Exec Interlude',
 'Super',
 'Dreams of Dai',
 '90s Love',
 'Youth',
 'The Eve',
 'AYAYAYA',
 'A Rainy Dawn',
 'Secret',
 'Spirit of Ink',
 'PINKCHELLA',
 'Big Fish',
 'VENI VIDI VICI',
 'Autumn Ripples',
 'Butterfly',
 'Broken Melodies',
 'The Call of the Past',
 'Teddy Bear',
 'Blue']

# Imports
If you do not have the packages, use pip install

In [12]:
# %pip install gspread oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import copy

# Generator
No need to touch below. Just run and the conflict matrix will exist (hopefully)

**WARNING: the code is EXTREMELY sensitive to incorrect time strings. Either make sure to check edited responses do not have extra spaces, commas, etc, or make code more robust.**

In [51]:
# use creds to create a client to interact with the Google Drive API --------------------------------------------------
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)
spreadsheet = client.open(sheet_name)
responses_sheet = spreadsheet.worksheet(response_worksheet_name)

worksheet_names = ['cleaning conflicts matrix', 'cleaning excuses matrix', 'staging conflicts matrix', 'staging excuses matrix']
if named:
    tmp = [name_of_run+" "+nm for nm in worksheet_names]
    worksheet_names = tmp

if create_new_worksheet:
    # if the spreadsheet already exists, cannot write/create a new one by the same name (set create_new_worksheet = False to edit existing sheet)
    cleaning_conflicts_matrix_sheet, cleaning_excuses_matrix_sheet, staging_conflicts_matrix_sheet, staging_excuses_matrix_sheet = [spreadsheet.add_worksheet(title=nm, rows=400, cols=22) for nm in worksheet_names]
else:
    # make sure the name actually exists tho
    cleaning_conflicts_matrix_sheet, cleaning_excuses_matrix_sheet, staging_conflicts_matrix_sheet, staging_excuses_matrix_sheet = [spreadsheet.worksheet(nm) for nm in worksheet_names]

worksheets = [cleaning_conflicts_matrix_sheet, cleaning_excuses_matrix_sheet, staging_conflicts_matrix_sheet, staging_excuses_matrix_sheet]
[wk.clear() for wk in worksheets]

first_response_row = 2
last_response_row = len(responses_sheet.get_all_values()) # assumes that all the previous rows filled in

# codeeee --------------------------------------------------

responses_list_of_lists = responses_sheet.get_all_values()
times_str = "8:00 - 8:30 AM, 8:30 - 9:00 AM, 9:00 - 9:30 AM, 9:30 - 10:00 AM, 10:00 - 10:30 AM, 10:30 - 11:00 AM, 11:00 - 11:30 AM, 11:30 - 12:00 PM, 12:00 - 12:30 PM, 12:30 - 1:00 PM, 1:00 - 1:30 PM, 1:30 - 2:00 PM, 2:00 - 2:30 PM, 2:30 - 3:00 PM, 3:00 - 3:30 PM, 3:30 - 4:00 PM, 4:00 - 4:30 PM, 4:30 - 5:00 PM, 5:00 - 5:30 PM, 5:30 - 6:00 PM, 6:00 - 6:30 PM, 6:30 - 7:00 PM, 7:00 - 7:30 PM, 7:30 - 8:00 PM, 8:00 - 8:30 PM, 8:30 - 9:00 PM, 9:00 - 9:30 PM, 9:30 - 10:00 PM, 10:00 - 10:30 PM, 10:30 - 11:00 PM, 11:00 - 11:30 PM, 11:30 - 12:00 AM"
times = times_str.split(', ')


#sorry brain dead this is spaghetti code so sorry
cleaning_conflict_matrix = [[t] for t in times]
none = [None] * len(dances)
for row in cleaning_conflict_matrix:
    row.extend([None]*len(dances))
header = ['']
header.extend(dances)
header = [header]
header.extend(cleaning_conflict_matrix)
cleaning_conflict_matrix = header

dance_ind = {}
for dance in dances:
    dance_ind[dance] = cleaning_conflict_matrix[0].index(dance)

time_ind = {}
for i in range(1, len(cleaning_conflict_matrix)):
    time_ind[cleaning_conflict_matrix[i][0]] = i
    
cleaning_excuse_matrix = copy.deepcopy(cleaning_conflict_matrix)
staging_conflict_matrix = copy.deepcopy(cleaning_conflict_matrix)
staging_excuse_matrix = copy.deepcopy(cleaning_conflict_matrix)
    

for row_ind in range(first_response_row,last_response_row):
    row = responses_list_of_lists[row_ind]
    name = row[2]
    relevant_dances = row[4].split(", ")
    cleaning_conflict_times = row[5].split(", ")
    cleaning_excuse = row[6]
    staging_conflict_times = row[7].split(", ")
    staging_excuse = row[8]

    
    # print("the times", cleaning_conflict_times)
    for d in relevant_dances:
        for ct in cleaning_conflict_times:
            if ct == 'N/A':
                continue
            if not cleaning_conflict_matrix[time_ind[ct]][dance_ind[d]]:
                cleaning_conflict_matrix[time_ind[ct]][dance_ind[d]] = 0
                cleaning_excuse_matrix[time_ind[ct]][dance_ind[d]] = {
                    'names': [], 
                    'excuses': [],
                    }
            cleaning_conflict_matrix[time_ind[ct]][dance_ind[d]] += 1
            cleaning_excuse_matrix[time_ind[ct]][dance_ind[d]]['names'].append(name)
            cleaning_excuse_matrix[time_ind[ct]][dance_ind[d]]['excuses'].append(cleaning_excuse)
            
        for st in staging_conflict_times:
            if st == 'N/A':
                continue
            #print("LOOKING AT ", st, "IN", time_ind)
            if not staging_conflict_matrix[time_ind[st]][dance_ind[d]]:
                staging_conflict_matrix[time_ind[st]][dance_ind[d]] = 0
                staging_excuse_matrix[time_ind[st]][dance_ind[d]] = {
                    'names': [], 
                    'excuses': [],
                    }
            staging_conflict_matrix[time_ind[st]][dance_ind[d]] += 1
            staging_excuse_matrix[time_ind[st]][dance_ind[d]]['names'].append(name)
            staging_excuse_matrix[time_ind[st]][dance_ind[d]]['excuses'].append(staging_excuse)
            
for r, row in enumerate(cleaning_excuse_matrix):
    for c, entry in enumerate(row):
        if not entry:
            cleaning_excuse_matrix[r][c] = ''
        else:
            cleaning_excuse_matrix[r][c] = str(entry)
            
for r, row in enumerate(staging_excuse_matrix):
    for c, entry in enumerate(row):
        if not entry:
            staging_excuse_matrix[r][c] = ''
        else:
            staging_excuse_matrix[r][c] = str(entry)
    
cleaning_conflicts_matrix_sheet.update('A1:Z35', cleaning_conflict_matrix)
cleaning_excuses_matrix_sheet.update('A1:Z35', cleaning_excuse_matrix)
staging_conflicts_matrix_sheet.update('A1:Z35', staging_conflict_matrix)
staging_excuses_matrix_sheet.update('A1:Z35', staging_excuse_matrix)

    
    

time strings ['8:00 - 8:30 AM', '8:30 - 9:00 AM', '9:00 - 9:30 AM', '9:30 - 10:00 AM', '10:00 - 10:30 AM', '10:30 - 11:00 AM', '11:00 - 11:30 AM', '11:30 - 12:00 PM', '12:00 - 12:30 PM', '12:30 - 1:00 PM', '1:00 - 1:30 PM', '1:30 - 2:00 PM', '2:00 - 2:30 PM', '2:30 - 3:00 PM', '3:00 - 3:30 PM', '3:30 - 4:00 PM', '4:00 - 4:30 PM', '4:30 - 5:00 PM', '5:00 - 5:30 PM', '5:30 - 6:00 PM', '6:00 - 6:30 PM', '6:30 - 7:00 PM', '7:00 - 7:30 PM', '7:30 - 8:00 PM', '8:00 - 8:30 PM', '8:30 - 9:00 PM', '9:00 - 9:30 PM', '9:30 - 10:00 PM', '10:00 - 10:30 PM', '10:30 - 11:00 PM', '11:00 - 11:30 PM', '11:30 - 12:00 AM']
[['', 'Exec Interlude', 'Super', 'Dreams of Dai', '90s Love', 'Youth', 'The Eve', 'AYAYAYA', 'A Rainy Dawn', 'Secret', 'Spirit of Ink', 'PINKCHELLA', 'Big Fish', 'VENI VIDI VICI', 'Autumn Ripples', 'Butterfly', 'Broken Melodies', 'The Call of the Past', 'Teddy Bear', 'Blue'], ['8:00 - 8:30 AM', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

C:\Users\jiefi\AppData\Local\Temp\ipykernel_44108\3494155121.py:127: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  cleaning_conflicts_matrix_sheet.update('A1:Z35', cleaning_conflict_matrix)
C:\Users\jiefi\AppData\Local\Temp\ipykernel_44108\3494155121.py:128: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  cleaning_excuses_matrix_sheet.update('A1:Z35', cleaning_excuse_matrix)
C:\Users\jiefi\AppData\Local\Temp\ipykernel_44108\3494155121.py:129: DeprecationWarn

{'spreadsheetId': '10KVCyJiD4HxJVoHL2LxpJRX_ghexP57DQYIX1k08wqg',
 'updatedRange': "'(edited) staging excuses matrix'!A1:T33",
 'updatedRows': 33,
 'updatedColumns': 20,
 'updatedCells': 660}